In [1]:
#importing packages 
import signal
import codecs
import serial
import os
import sys
import time
from time import sleep
# import Cryocode
import serial
from serial import SerialException

import serial.tools.list_ports
from datetime import datetime,timedelta
import glob
import numpy as np
import pandas as pd
%matplotlib inline

from lakeshore import Model350
from time import sleep
from IPython.display import clear_output
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import time
import plotly
%matplotlib inline

# """Adding Asthetics to the plot"""
# plt.rc('font', size=15)          # controls default text sizes
# plt.rc('axes', titlesize=20)     # fontsize of the axes title
# plt.rc('axes', labelsize=15)     # fontsize of the x and y labels
# plt.rc('xtick', labelsize=15)    # fontsize of the tick labels
# plt.rc('ytick', labelsize=15)    # fontsize of the tick labels
# plt.rc('legend', fontsize=15)    # legend fontsize
# %config InlineBackend.figure_format = 'png'
# plt.rcParams['figure.figsize'] = [24, 12]  # set plotsize




In [2]:
def lakeshore_initialize():
    try: 
        myinstrument =Model350()
        print("connection successful")
        return(myinstrument)
    except: 
        print("Device not found")
        

def get_temp_data():
    import datetime
    data = {
        'tempA_log': [],
        'tempB_log': [],
        'tempC_log': [],
        'tempD_log': [],
        'time':[]
        }
        
    # data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempA_log'].append(np.float32(myinstrument.query('KRDG? A')))
    time.sleep(0.1)
    data['tempB_log'].append(np.float32(myinstrument.query('KRDG? B')))
    time.sleep(0.1)
    data['tempC_log'].append(np.float32(myinstrument.query('KRDG? C')))
    time.sleep(0.1)
    data['tempD_log'].append(np.float32(myinstrument.query('KRDG? D')))
    data['time'].append(datetime.datetime.now())
    return data
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df
def log_temp_data_step_duration(step,duration):
    import datetime
    start_time=datetime.datetime.now()
    log_duration = float(duration) #seconds
    log_step=float(step) #seconds 
    df = pd.DataFrame(get_temp_data())
    while datetime.datetime.now()-start_time<datetime.timedelta(seconds=log_duration):
        # print("its true")
        df2=pd.DataFrame(get_temp_data())
        df=df.append(df2,ignore_index = True)
        sleep(log_step)
    return df

In [13]:
def start_connection(cryo_port):
    """Start Serial connection with Cryocooler at port=port"""
    def open_port(cryo_port): 
        try:
            
            ser0 = serial.Serial(port=cryo_port,
                                baudrate=4800, #9600bits/s number of data bits
                                parity=serial.PARITY_NONE, #parity checks if the bit sent is odd or even
                                stopbits=serial.STOPBITS_ONE, #signal is high or low
                                bytesize=serial.EIGHTBITS,
                                timeout=1)
        except SerialException:
            return None
        else: 
            return ser0  
    ser0=open_port(cryo_port)
    while ser0 == None: 
        print("Serial port busy. Waiting to connect")
        time.sleep(2)
        ser0=open_port(cryo_port)
    else: 
        print("Connected")
        return(ser0)

# def start_connection(port):
#     """Start Serial connection with Cryocooler at port=port"""
    
#     try: 
#         ser0  = serial.Serial(port=port, #common port for Windows systems,
#                             baudrate=4800, #9600bits/s number of data bits
#                             parity=serial.PARITY_NONE, #parity checks if the bit sent is odd or even
#                             stopbits=serial.STOPBITS_ONE, #signal is high or low
#                             bytesize=serial.EIGHTBITS,
#                             timeout=1) #size of bit sent(can be 5,6,7,8,9)
#         msg="Cryocooler connected\n"
#         # print(msg)
#         return ser0
#     except: 
#         msg="Error Connection not estabilished"
#         return msg
#     finally:
#         msg=msg+"Close the connection before exiting program"
#         print(msg)
#         #ser0.close()
        

def stop_connection(ser): 
    try: 
       ser.close()
       print("Connection stopped")
    except: 
       print("Something went wrong")
    return 

def get_temperature(ser0):
    import datetime
    """Cold Tip Temperature: Display Cold-Tip Temperature"""
    data = {
    'cryo_temp': [],
    'time':[]}
    #while True: 
    ser0.write(b'TC\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    cmd = out[0].decode('utf-8').strip()
    temperature = out[1].decode('utf-8').strip()
    data['cryo_temp']=float(temperature)
    data['time']=datetime.datetime.now()
    return data

def version(): 
    ser0.write(b'VERSION\r')
    time.sleep(1)
    out =ser0.readlines();
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,str(out)

def serial_number(): 

    ser0.write(b'SERIAL\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def get_mode(): 
    """Display Configured Cryocooler Type
        Soft-Stop Control Mode: Set Soft-Stop Control Mode: SET SSTOPM=<Value>
        000.00 = Sunpower reserved mode
        001.00 = CryoTel CT
        002.00 = CryoTel GT
        003.00 = CryoTel MT
        
        For Hamden lab this is always set to 3.00 and never changed as we have only the MT cryocooler
    """
    
    ser0.write(b'MODE\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd, out

def get_pid_mode(): 
    """Display Current Control Mode: SET PID
        Use the SET PID command to display the controller control mode that is currently in use.
        A return of 000.00 indicates that power control mode is currently in use.
        A return of 002.00 indicates that temperature control mode is currently in use.
    """
    ser0.write(b'SET PID\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def set_pid_mode(value): 
    """Display Current Control Mode: SET PID = <Value>
        Use the SET PID=<Value> command to change the control mode that’s currently being used.
        Entering a value of 0 changes the control mode currently in use to the power control mode. In this mode, the con-troller will supply power to the cryocooler at the target power level set by the user. (You can use the SET PWROUT=<Value> command to change the target power output setting.)
        Entering a value of 2 changes the control mode currently in use to the temperature control mode. In this mode, the controller will try to maintain the cold tip at the target temperature setting. (You can use the SET TTARGET=<Value> command to change the target temperature setting.)
        The control mode will revert to the default control mode upon cycling the power. (You can use the SAVE PID command to change the default control mode to the control mode that’s currently in use.)

    """
    cmdstr= 'SET PID='+str(value)+'\r'
    cmd=bytes(cmdstr,'UTF-8')
    ser0.write(cmd)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def save_pid_mode(): 
    """Change Default Control Mode: SAVE PID
        Use the SAVE PID command to save the control mode that is currently in use as the new default control mode. The factory-set default control mode is temperature control mode. (You can use the SET PID command to dis-play the control mode that is currently in use, and you can use the SET PID=<Value> command to specify which control mode to use.) The control mode that you save with this command will be the control mode used after the system is power cycled.
        A return of 000.00 indicates that power control mode has been saved as the default control mode. A return of 002.00 indicates that temperature control mode has been saved as the default control mode.
    """
    ser0.write(b'SAVE PID')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def factory_reset():
    """Controller Parameters: Reset Controller Parameters to Factory Defaults
    Use the RESET=F command to reset the controller parameters to their factory defaults as listed in table B-3 of the manual.
    """
    ser0.write(b'RESET=F\r')
    time.sleep(1)
    out =ser0.readlines()
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def get_cooler_power():
    """Display Cooler Power as Measured by Controller: P
    Cooler power as measured by the controller is how much electrical power in watts the cryocooler is using as measured by the controller. (This is not necessarily the same as the power level that the controller is command-ing as displayed by the E command.)
    
    """
    ser0.write(b'P\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out


def get_commdanded_power_limits():
    """Display Current Commanded Power & Power Limits: E
        The current commanded power is current power output that the controller is commanding to be supplied to the cryocooler. The current power limits are the minimum and maximum controller power output levels that the con-troller calculates to be the appropriate limits based on the current conditions in the system.
        
        Use the E command to display the current commanded power and the power limits.
        The first line of the values returned by this command is the maximum allowable power output to the cryocooler for the current cold tip temperature. This number will be either the maximum safe power output, or the user-de-fined maximum power output, whichever is lower. (The maximum safe power output to the cryocooler is a func-tion of the cold tip temperature and increases as the cold tip temperature decreases [see figure 2-6]. The maximum safe power output at 77 K is 165 W.)
        The second line of the values returned by this command is the minimum allowable power output to the cry-ocooler. This number will be either the minimum safe power output, or the user-defined minimum power output, whichever is higher. (The minimum safe power output to the cryocooler is a function of the cold tip temperature, but is steady at 60 W over most of the operating range [see figure 2-6]. The minimum safe power output at 77 K is 60 W.)
        The third line of the values returned by this command is the current commanded power (not necessarily the target power output set for power control mode or the power the cryocooler is using as displayed by the P command). All values are in watts.
    """
    
    ser0.write(b'E\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def error_code():
    """Display Current Commanded Power & Power Limits: ERROR)
    An error code is a six-digit binary number that identifies which, if any, errors are occurring. The position of the 1 within the error code determines what error is being reported (table B-4 in the manual or below). Multiple errors are indicated by multiple 1s in the error code, each 1 representing a different error. An error code consisting of six zeros means there are no errors.
    -----------
    Error Codes: 
        000001- Over Current
        000010- Jumper Error
        000100- Serial Communication Error
        001000- Non-Volatile Memory Error
        010000- Watchdog Error
        100000- Temperature Sensor Error
    """
    ser0.write(b'ERROR\r')
    time.sleep(1)
    out =ser0.readlines()
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out
    

def set_integralconstant(value):
    """ Set Integral Constant of Temperature Control Loop: SET KI=<Value>
    In some applications, advanced users experienced in tuning PID control loops may want to modify the PI constants to tune the temperature control loop to better match the dynamics of their individual system. The default integral constant for the MT cryocooler is 1.0.
    
    """
    cmdstr= 'SET KI='+str(value)+'\r'
    cmd_b =cmdstr.encode('utf-8')
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    cmd = out[0].decode('utf-8').strip()
    out = out[1].decode('utf-8').strip()
    return cmd,out

def get_integralconstant():
    """Display Integral Constant of Temperature Control Loop: SET KI""" 
    ser0.write(b'SET KI\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def state():
    """Display Operating State:STATE
        Use the STATE command to display a list of the controller’s operating parameters (as described in table A-5) and their current values. All of the individual parameter values can also be displayed using the corresponding individ-ual parameter display commands listed elsewhere in this appendix. This command is useful when troubleshooting the system.
            
        The operating state of the controller is defined as the current value of the controller's operating parameters which are listed and described in table A-5. The factory default values for these parameters (except TSTAT) are listed in table A-3.
        
         Cryocooler Model Designation
            TSTATM          Thermostat Mode
            TSTAT           Thermostat State
            SSTOPM          Soft-Stop Control Mode
            SSTOP           Soft-Stop State
            PID             Control Mode Currently in Use
            LOCK            User Lock State
            MAX             User-Defined Maximum Power Output
            MIN             User-Defined Minimum Power Output
            PWOUT           Target Power Output
            TTARGET         Target Temperature
            TBAND           Temperature Band
            TEMP KP         Proportional Constant of Temperature Control Loop
            TEMP KI         Integral Constant of Temperature Control Loop
    """
    ser0.write(b'STATE\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_proportionalconstant(value):
    """Set Integral Constant of Temperature Control Loop: SET KI=<Value>
        Use the SET KP=<Value> command to set the proportional constant of the temperature control loop.
    """
    cmdstr= 'SET KP='+str(value)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def get_proportionalconstant():
    """Display Proportional Constant of Temperature Control Loop: SET KP
        Use the SET KP=<Value> command to set the proportional constant of the temperature control loop.

    """ 
    ser0.write(b'SET KP\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out


"""Soft-Stop Control Mode
Set Soft-Stop Control Mode: SET SSTOPM=<Value>
Display Soft-Stop Control Mode: SET SSTOPM"""

def get_soft_stop_mode():
    """
    Soft-Stop Control Mode
    Soft-Stop Control Mode: Display Soft-Stop Control Mode: SET SSTOPM

    The soft-stop control mode identifies the method that has been selected to enable and disable the soft-stop func-tion. There are two methods: one is to use the SET SSTOP=<Value> command; the other is to use Digital In-put 1 (pin 5 on the controller’s I/O connector). In the Digital Input 1 method, setting the input high enables the soft-stop function. The Onboard Isolated 5 V output on pin 10 or 12 on the controller’s I/O connector can be used to set the input high. Setting the input low or disconnecting it disables the soft-stop function.
    
    """
    ser0.write(b'SET SSTOPM\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_soft_stop_mode(value=3):
    """
    Soft-Stop Control Mode
    Set Soft-Stop Control Mode" SET SSTOP=<Value>
    
    Description: 
    Use the SET SSTOP=<Value> command to begin a soft stop of the cryocooler, or to start the cryocooler from its stopped state. A soft stop is the recommended shutdown method in which the controller slowly ramps down operation of the cryocooler and parks the piston against the end stop before shutting it down completely. This re-duces vibration, allows the controller to remain on, and reduces the risk of self-excitation (when turning off the cryocooler at very low temperatures–below 77 K). This command will only work if the soft-stop control mode is set to mode 0. (If set to mode 1, then Digital Input 1 is used to control the soft-stop state.)
    Usage: 
    Entering a value of 0 restarts a stopped cryocooler and disables the soft-stop function.
    Entering a value of 1 begins a soft stop of the cryocooler and enables the soft-stop function. Do not remove power from the controller until the shutdown process has been completed and the word COMPLETE is displayed on the computer screen.
    
    
    
    """
    if value==3: 
        cmdstr= 'SET SSTOPM='+str(value)+'\r'
        cmd_b= cmdstr.encode(cmdstr)
        ser0.write(cmd_b)
        time.sleep(1)
        out =ser0.readlines()
    else: 
        print("Are you trying to set the controller to operate with MT? Then use mode 3. For details refer to the manual")
    #print(out)
    #print("Command Sucessful")
    return out

def get_soft_stop_state():
    """Soft-Stop Control Mode
        Display Soft-Stop Control Mode: SET SSTOPM
        
        Use the SET SSTOP command to display the current soft-stop state, i.e., whether the soft-stop function is ena-bled or disabled.
        A return of 000.00 indicates that the soft-stop function is disabled.
        A return of 001.00 indicates that soft-stop function is enabled.
    """
    ser0.write(b'SET SSTOP\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_soft_stop_state(value):
    """
        The soft-stop state indicates whether the soft-stop function is enabled or disabled. With the soft-stop function ena-bled, the cryocooler is stopped, or is being ramped down, parked, and stopped, and will not start up when power is cycled to the controller. With the soft-stop function disabled, the cryocooler is in the process of starting up or is running, and will start up normally when power to the controller is cycled.
        Description: 
        Use the GET SSTOP=<Value> command to begin a soft stop of the cryocooler, or to start the cryocooler from its stopped state. A soft stop is the recommended shutdown method in which the controller slowly ramps down operation of the cryocooler and parks the piston against the end stop before shutting it down completely. This re-duces vibration, allows the controller to remain on, and reduces the risk of self-excitation (when turning off the cryocooler at very low temperatures–below 77 K). This command will only work if the soft-stop control mode is set to mode 0. (If set to mode 1, then Digital Input 1 is used to control the soft-stop state.)
        Usage: 
        Entering a value of 0 restarts a stopped cryocooler and disables the soft-stop function.
        Entering a value of 1 begins a soft stop of the cryocooler and enables the soft-stop function. Do not remove power from the controller until the shutdown process has been completed and the word COMPLETE is displayed on the computer screen.
    """
    
    cmdstr= 'SET SSTOP='+str(value)+'\r'
    cmd=bytes(cmdstr,'UTF-8')
    ser0.write(cmd)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    
    return out

def get_target_power_output():
    """Target Power Output
        Use the SET PWOUT command to display the target power output setting.
        Display Target Power Output: SET PWOUT
        
        The target power output setting is the power level that the user has set as the target power output to send to the cryocooler when in power control mode. This setting is only a target power level: the actual power output that the controller is commanding is displayed by the E command and is subject to the power limits also displayed by the E command. The default target power output setting is 0 W.
        While any number from 0.0 to 999.99 can be entered as the target power output, the controller will not command a power level that is outside of the currently-calculated allowable operating power range of the cryocooler (as shown using the E command) regardless of the target power output set by the user. Refer to the E command sec-tion for details on how the allowable operating power range is calculated.
    
    """
    ser0.write(b'SET PWOUT\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_target_power_output(value):
    """Target Power Output
        Use the SET PWOUT=<Value> command to change the target power output setting.
        Display Target Power Output: SET PWOUT=<value>
        
        The target power output setting is the power level that the user has set as the target power output to send to the cryocooler when in power control mode. This setting is only a target power level: the actual power output that the controller is commanding is displayed by the E command and is subject to the power limits also displayed by the E command. The default target power output setting is 0 W.
        While any number from 0.0 to 999.99 can be entered as the target power output, the controller will not command a power level that is outside of the currently-calculated allowable operating power range of the cryocooler (as shown using the E command) regardless of the target power output set by the user. Refer to the E command sec-tion for details on how the allowable operating power range is calculated.
    """
    cmdstr= 'SET PWOUT='+str(value)+'\r'
    # cmd_b= cmdstr.encode(cmdstr)
    cmd=bytes(cmdstr,'UTF-8')
    ser0.write(cmd)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out


def get_target_temperature():
    
    """Target Temperature: Get Target Temperature: SET TTARGET
        Use the SET TTARGET=<Value> command to set the target temperature.
        The target temperature is the target cold tip temperature that the controller will try to attain, ±0.1 K, when the controller is in temperature control mode. The default target temperature is 77 K.
    
    """
    ser0.write(b'SET TTARGET\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_target_temperature(value):
    """Target Temperature: Set Target Temperature: SET TTARGET
        Use the SET TTARGET command to display the target temperature.
        The target temperature is the target cold tip temperature that the controller will try to attain, ±0.1 K, when the controller is in temperature control mode. The default target temperature is 77 K.
    """
    cmdstr= 'SET TTARGET='+str(value)+'\r'
    cmd=bytes(cmdstr,'utf-8')
    ser0.write(cmd)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def get_target_band():
    """Get Temperature Band: SET TBAND
        Use the SET TBAND command to display the ± temperature band in kelvins.
        The temperature band is the ± tolerance band used with the target temperature to determine when the cold tip is considered to be at the target temperature (for the sole purposes of operating the controller’s LEDs and the Digital Out 4 pin on the controller’s I/O connector). For example, if the target temperature is set at 77 K, and the ± tem-perature band is set at ±0.5 K, and the controller is in temperature control mode, then the green LED will be on and the Digital Out 4 pin will be high (5 V) when the cold tip temperature is 77 K ±0.5 K. Also, the red LED will be on and the Digital Out 4 pin will be low (0 V) when the cold tip temperature is outside of that range. The de-fault ± temperature band is ±0.5 K.
    
    """
    ser0.write(b'SET TBAND\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_target_band(band):
    """Set Temperature Band: SET TBAND=<Value>
        Use the SET TBAND=<Value> command to set the ± temperature band in kelvins.
        The temperature band is the ± tolerance band used with the target temperature to determine when the cold tip is considered to be at the target temperature (for the sole purposes of operating the controller’s LEDs and the Digital Out 4 pin on the controller’s I/O connector). For example, if the target temperature is set at 77 K, and the ± tem-perature band is set at ±0.5 K, and the controller is in temperature control mode, then the green LED will be on and the Digital Out 4 pin will be high (5 V) when the cold tip temperature is 77 K ±0.5 K. Also, the red LED will be on and the Digital Out 4 pin will be low (0 V) when the cold tip temperature is outside of that range. The de-fault ± temperature band is ±0.5 K.
    """
    cmdstr= 'SET TBAND='+str(band)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out


def get_thermostat_mode():
    """Get Thermostat Mode: Display Thermostat Mode: SET TSTATM
        The thermostat mode indicates whether the controller is set to work with a thermostat (mode 1), or is set to not work with a thermostat (mode 0). When in mode 1, a thermostat connected across pin 7 (Digital Input 3) and pin 10 or 12 (Onboard Isolated 5 V Output) of the controller’s I/O connector can be used to shut down the cryocooler when the temperature is outside of a desired temperature range. (As long as pin 7 is high [thermostat closed], a running cryocooler is allowed to continue running, but anytime the thermostat becomes open, the controller shuts down the cryocooler. At that point, the thermostat must be closed before the cryocooler can be started up again. This does not generate an error and does not require a power cycle to restart.)
        

    """
    ser0.write(b'SET TSTATM\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_thermostat_mode(mode):
    """Set Thermostat Mode: SET Thermostat Mode: SET TSTATM=<Mode>
        Use the SET TSTATM=<Value> command to set the thermostat mode of the controller.

        The thermostat mode indicates whether the controller is set to work with a thermostat (mode 1), or is set to not work with a thermostat (mode 0). When in mode 1, a thermostat connected across pin 7 (Digital Input 3) and pin 10 or 12 (Onboard Isolated 5 V Output) of the controller’s I/O connector can be used to shut down the cryocooler when the temperature is outside of a desired temperature range. (As long as pin 7 is high [thermostat closed], a running cryocooler is allowed to continue running, but anytime the thermostat becomes open, the controller shuts down the cryocooler. At that point, the thermostat must be closed before the cryocooler can be started up again. This does not generate an error and does not require a power cycle to restart.)
    
        Entering a value of 0 (mode 0) disables the controller's ability to work with a thermostat.
        Entering a value of 1 (mode 1) enables the controller's ability to work with a thermostat.
    
    """

    cmdstr= 'SET TSTATM='+str(mode)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out


def get_thremostat_state():
    """Get Thermostat STATE: Display Thermostat STATE: SET TSTAT
        Use the TSTAT command to display the current state of the thermostat.

        The thermostat state indicates the current state (open or closed) of a thermostat connected across pin 7 and pin 10 or 12 of the controller’s I/O connector. This state is only meaningful if thermostat mode has been enabled (using the SET TSTATM=1 command) and a thermostat is installed.
        If the thermostat state is open, that means the cryocooler has been automatically shut down or is in the process of automatically shutting down. The thermostat must then be closed before the cryocooler can be started up again. Startup will happen automatically when the thermostat closes.

        A return of 000.00 indicates that the thermostat is open.
        A return of 001.00 indicates that the thermostat is closed.
    
    """
    ser0.write(b'SET TSTAT\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

"""User-Defined Minimum & Maximum Power Output
Set User-Defined Minimum Power Output: SET MIN=<Value>
Display User-Defined Minimum Power Output: SET MIN
Set User-Defined Maximum Power Output: SET MAX=<Value>

Display User-Defined Maximum Power Output: SET MAX
Display User-Defined Minimum & Maximum Power Output: SHOW MX"""

def get_user_min_power():
    
    """Set Thermostat Mode: Display Thermostat Mode: SET TSTATM
        Use the SET MIN=<Value> command to set the user-defined minimum power output setting for the controller.
        
        The user-defined minimum power output setting for the controller is used in both the power control mode and the temperature control mode. Entering a value that is less than the minimum safe power output to the cryocooler (60 W at 77 K) will not result in damage to the cryocooler because the controller will not command a power that is less than the minimum safe power output to the cryocooler. The minimum power output to the cryocooler will either be the minimum safe power output, or the user-defined minimum power output, whichever is higher. The default user-defined minimum power output setting is 0 W.
        The user-defined maximum power output setting for the controller is used in both the power control mode and the temperature control mode. Entering a value that exceeds the maximum safe power output to the cryocooler will not result in damage to the cryocooler because the controller will not command a power that exceeds the maxi-mum safe power output to the cryocooler. The maximum safe power output to the cryocooler is a function of the cold tip temperature and increases as the cold tip temperature decreases. (It’s 165 W at 77 K.) The maximum power output to the cryocooler will be either the maximum safe power output, or the user-defined maximum power output, whichever is lower. The default user-defined maximum power output setting is 300 W.
    """
    ser0.write(b'SET MIN\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def get_user_max_power():
    """Set Thermostat Mode: Display Thermostat Mode: SET TSTATM
        Use the SET MAX command to display the user-defined maximum power output setting for the controller.
        
        The user-defined minimum power output setting for the controller is used in both the power control mode and the temperature control mode. Entering a value that is less than the minimum safe power output to the cryocooler (60 W at 77 K) will not result in damage to the cryocooler because the controller will not command a power that is less than the minimum safe power output to the cryocooler. The minimum power output to the cryocooler will either be the minimum safe power output, or the user-defined minimum power output, whichever is higher. The default user-defined minimum power output setting is 0 W.
        The user-defined maximum power output setting for the controller is used in both the power control mode and the temperature control mode. Entering a value that exceeds the maximum safe power output to the cryocooler will not result in damage to the cryocooler because the controller will not command a power that exceeds the maxi-mum safe power output to the cryocooler. The maximum safe power output to the cryocooler is a function of the cold tip temperature and increases as the cold tip temperature decreases. (It’s 165 W at 77 K.) The maximum power output to the cryocooler will be either the maximum safe power output, or the user-defined maximum power output, whichever is lower. The default user-defined maximum power output setting is 300 W.
    """
    ser0.write(b'SET MAX\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def get_user_min_max_power():
    """Set Thermostat Mode: Display Thermostat Mode: SET TSTATM
        Use the SHOW MX command to display the user-defined minimum and maximum power output settings for the controller. (You can also use the SET MIN and SET MAX commands to display the same settings individually.)
        The first line of the values returned by this command is the used-defined minimum power output setting for the controller. The second line of the values returned by this command is the used-defined maximum power output setting for the controller. All values are in watts.
    """
    ser0.write(b'SHOW MX\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

"""User-Lockable Commands
Lock User-Lockable Commands: LOCK=<Password>
Unlock User-Lockable Commands: UNLOCK=<Password>
Set User Password: SET PASS=<Value>
Display User Lock State: LOCK

Commands in the user-lockable commands group (table B-1) can be locked so that the parameters associated with those commands cannot be changed. To lock the commands, you must enter a user password. In order to unlock a command, you must unlock the entire group of commands by entering the user password. The default user pass-word is STIRLING, but you can change the user password when the commands are unlocked. The controller is shipped with these commands in the unlocked state. If the commands are locked and the user password has been misplaced, the controller must be sent back to Sunpower so that we can unlock the commands.

"""

def get_lock_state():
    """Display User Lock State: LOCK
        Use the LOCK command to display the user lock state.
        A return of 000.00 indicates that the commands are unlocked.
        A return of 001.00 indicates that the commands are locked.
    """
    ser0.write(b'LOCK\r')
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def lock_ud_commands(passwd):
    """Lock User-Lockable Commands: LOCK=<Password>
        Use the LOCK=<Password> command to lock access to the commands in the user-lockable commands group (table B-1). “<Password>” means “type the currently-set password”.
    
    """
    cmdstr= 'LOCK='+str(passwd)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def unlock_ud_commands(passwd):
    """UNLock User-Lockable Commands: UNLOCK=<Password>
        Use the UNLOCK=<Password> command to unlock access to the commands in the user-lockable commands group (table B-1). “<Password>” means “type the currently-set password”.
        A return of 000.00 confirms that the commands are unlocked.
    """
    cmdstr= 'UNLOCK='+str(passwd)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def set_password(passwd):
    """Set User Password: SET PASS=<Value>
        Use the SET PASS=<Value> command to define a user password.
        Enter an alphanumeric value between 1 and 10 characters in length. This value will replace the previous password.
        A return of 001.00 confirms that the password was successfully changed.
    """
    cmdstr= 'SET PASS='+str(passwd)+'\r'
    cmd_b= cmdstr.encode(cmdstr)
    ser0.write(cmd_b)
    time.sleep(1)
    out =ser0.readlines()
    #print(out)
    #print("Command Sucessful")
    return out

def cryo_temp_single_read(ser0):
    import datetime
    out = get_temperature(ser0)
    cryo_temp_df=pd.DataFrame(data=[out])
    return cryo_temp_df
def cryo_temp_log_duration(duration=5):
    #log temperature data for {duration} seconds
    import datetime
    start_log=datetime.datetime.now()
    elapsed_time=datetime.datetime.now()-start_log
    cryo_temp_df =pd.DataFrame(data=None)
    out = get_temperature()
    cryo_temp_df=cryo_temp_df.append(pd.DataFrame([out[1:]]))
    while elapsed_time.seconds<duration: 
        out = get_temperature()
        cryo_temp_df=cryo_temp_df.append(pd.DataFrame([out[1:]],columns={"time","temp_cryo"}))
        elapsed_time=datetime.now()-start_log       
    return cryo_temp_df

In [4]:
cooldown_power=30 #W
det_offset_cf1=22.485 #K
det_offset_coldtip=51 #K  
detector_temperature=273-90 #-90C
cf1_temperature=detector_temperature-det_offset_cf1
coldtip_temperature = 132
switchover_offset=20

cryo_temp_steps= np.array([200,160,coldtip_temperature])
cryo_temp_switchover=cryo_temp_steps+int(switchover_offset)
det_temp_steps=cryo_temp_steps+int(det_offset_coldtip)
cf1_temp_steps=cryo_temp_steps-int(det_offset_cf1)

# modeswitch_temperature=160
# set_pid_mode(0)
# set_target_power_output(cooldown_power)
# set_soft_stop_state(0)1

print(f"Intial cooldown power set to {cooldown_power} W")
print(f"Target Detector Temperature: {detector_temperature} K")
# print(f"Target Cold tip temperature will be set to {coldtip_temperature} K after Mode switches to Temp mode at {modeswitch_temperature}")
# print(f"Mode switch temperateure: {modeswitch_temperature}")
print(f"Crycooler tip Temperature steps= {cryo_temp_steps} K")
print(f"CF1 Temperature steps= {cf1_temp_steps} K")
print(f"Detector Temperature steps= {det_temp_steps} K")

print(f"Target CF1 Temperature in lakeshore: {cf1_temperature}")
print(f"Target cold tip Temperature: {coldtip_temperature}")



# cooldown_power=30 #W
# det_offset_cf1=22.485 #K
# det_offset_coldtip=51 #K  
# detector_temperature=273-90 #-90C
# cf1_temperature=detector_temperature-det_offset_cf1
# coldtip_temperature = 132
# modeswitch_temperature=160
# # set_pid_mode(0)
# # set_target_power_output(cooldown_power)
# # set_soft_stop_state(0)
# print(f"Intial cooldown power set to {cooldown_power} W")
# print(f"Target Detector Temperature: {detector_temperature} K")
# print(f"Target Cold tip temperature will be set to {coldtip_temperature} K after Mode switches to Temp mode at {modeswitch_temperature}")
# print(f"Mode switch temperateure: {modeswitch_temperature}")
# print(f"Target CF1 Temperature in lakeshore: {cf1_temperature}")



Intial cooldown power set to 30 W
Target Detector Temperature: 183 K
Crycooler tip Temperature steps= [200 160 132] K
CF1 Temperature steps= [178 138 110] K
Detector Temperature steps= [251 211 183] K
Target CF1 Temperature in lakeshore: 160.515
Target cold tip Temperature: 132


connection successful


In [14]:
cryo_port='COM7'

today=datetime.now()
# filename_base=today.strftime("%m_%d_%Y")+'_1'

from IPython.display import clear_output
# import datetime
import pandas as pd
import time 

# cooling =True
Restart=0#Raise flag if the logging is being restarted and the data in the exisiting dataframe must not be re written
Sensor_map={'LSA':'LS Input A: EMCCD Detector',
            'LSB':'LS Input B: Cold Finger 1',
            'LSC':'LS Input C: Cold Finger 2',
            'LSD':'LS Input D: Vapour Shroud',
            'Cryo':'Cryo Cold Tip'
            }

if Restart==0: #if the logging is not being restarted .
    #Get Lakeshore temperature data for the first time
    filename_base=today.strftime("%m_%d_%Y")
    myinstrument.device_serial.open()
    df=pd.DataFrame(get_temp_data())
    myinstrument.device_serial.close()    
    #Get cryo temperature data for the first time
    ser0=start_connection(cryo_port)
    df_cryo = cryo_temp_single_read(ser0)
    stop_connection(ser0)

start_time=datetime.now()
cryo_path=f"Cryocooler_Templog_{filename_base}.csv"
lakeshore_path=f"Lakeshore_Templog_{filename_base}.csv"
webpath =f"Exp_Templog_{filename_base}.html"
# Use this function to search for any files which match your filename
files_present = glob.glob(cryo_path)+glob.glob(lakeshore_path)
# if no matching files, write to csv, if there are matching files, print statement
if files_present:
       df_cryo=pd.read_csv(cryo_path)
       df=pd.read_csv(lakeshore_path)

log_duration = 24*60*60 #seconds
log_step=1 #seconds 
time_check_state=start_time

while datetime.now()-start_time<timedelta(seconds=log_duration):
    # print("its true")
    timer_save_data=datetime.now()
    timer_plot_data=timer_save_data
    
    while datetime.now()-timer_save_data<timedelta(seconds=60):
            
        myinstrument=lakeshore_initialize()
        dftemp=pd.DataFrame(get_temp_data())
        myinstrument.device_serial.close() 
        
        
        ser0=start_connection(cryo_port)
        dftemp_cryo=cryo_temp_single_read(ser0)
        stop_connection(ser0)

        df=df.append(dftemp,ignore_index = True)
        df_cryo=df_cryo.append(dftemp_cryo,ignore_index = True)
        #display(df[-5:])
        df_cryo.to_csv(cryo_path)
        df.to_csv(lakeshore_path)
        sleep(60)
        #clear_output(wait=True)    
       
    if datetime.now()-timer_plot_data>timedelta(seconds=60):
        clear_output(wait=True)
        df_read_cryo=pd.read_csv(cryo_path)
        df_read_lakeshore=pd.read_csv(lakeshore_path)
        # df=df.append(df_read_lakeshore,ignore_index=True)
        
        fig = plotly.subplots.make_subplots(rows=3, cols=2, vertical_spacing=0.2)
        fig['layout']['margin'] = {'l': 30, 'r': 10, 'b': 30, 't': 100}
        fig['layout']['legend'] = {'x': 0, 'y': -0.25, 'xanchor': 'left','orientation':"h"}
        
        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempA_log,
            'name': Sensor_map['LSA'],
            'type': 'scatter'
        }, 1, 1)

        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempB_log,
            'name': Sensor_map['LSB'],
            'type': 'scatter'
        }, 1, 1)

        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempC_log,
            'name': Sensor_map['LSC'],
            'type': 'scatter'
        }, 1, 1)

        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempD_log,
            'name': Sensor_map['LSD'],
            'type': 'scatter'
        }, 1, 1)
        
        fig.append_trace({
            'x': df_read_cryo.time,
            'y': df_read_cryo.cryo_temp,
            'name': Sensor_map['Cryo'],
            'type': 'scatter'
        }, 1, 1)
        
        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempA_log,
            'name': Sensor_map['LSA'],
            'type': 'scatter'
        }, 1, 2)


        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempB_log,
            'name': Sensor_map['LSB'],
            'type': 'scatter'
        }, 2, 1)

        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempC_log,
            'name': Sensor_map['LSC'],
            'type': 'scatter'
        }, 2, 2)

        fig.append_trace({
            'x': df_read_lakeshore.time,
            'y': df_read_lakeshore.tempD_log,
            'name': Sensor_map['LSD'],
            'type': 'scatter'
        }, 3, 1)
        
        fig.append_trace({
            'x': df_read_cryo.time,
            'y': df_read_cryo.cryo_temp,
            'name': Sensor_map['Cryo'],
            'type': 'scatter'
        }, 3, 2)
        # fig.update_layout(yaxis = dict(range=[77,300]))
        fig.update_layout(width=900,height=800)

        fig.update_yaxes(range=[70.0,300.0],row=1,col=1)
        fig.update_yaxes(range=[70,300],row=1,col=2)
        fig.update_yaxes(range=[70,300],row=2,col=1)
        fig.update_yaxes(range=[70,300],row=2,col=2)
        fig.update_yaxes(range=[70,300],row=3,col=1)
        
        fig.update_layout(
            title={
                'text':"Detector cooling chain test: Temperature vs time",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
        # fig.update_layout(xaxis_tickformat = '%H:%M')
        # fig.update_xaxes(ticks= "outside",
        #                  ticklabelmode= "period", 
        #                  tickcolor= "black", 
        #                  ticklen=10, 
        #                  dtick=10,
        #                 #  minor=dict(
        #                 #      ticklen=4,  
        #                 #      dtick=7*24*60*60*1000,  
        #                 #      tick0="2016-07-03", 
        #                 #      griddash='dot', 
        #                 #      gridcolor='white'),
        #                 row=1,col=1)
        fig.layout.template='simple_white'
        fig.show()
        fig.write_html(f"{webpath}")

AttributeError: module 'datetime' has no attribute 'now'

In [16]:
cryo_port='COM7'
import datetime 
from datetime import timedelta

today=datetime.datetime.now()
# filename_base=today.strftime("%m_%d_%Y")+'_1'

from IPython.display import clear_output
# import datetime
import pandas as pd
import time 
Restart=0#Raise flag if the logging is being restarted and the data in the exisiting dataframe must not be re written

# cooling =True
Sensor_map={'LSA':'LS Input A: EMCCD Detector',
            'LSB':'LS Input B: Cold Finger 1',
            'LSC':'LS Input C: Cold Finger 2',
            'LSD':'LS Input D: Vapour Shroud',
            'Cryo':'Cryo Cold Tip'
            }

Color_map={'LSA':px.colors.qualitative.Safe[0], 
            'LSB':px.colors.qualitative.Safe[1],
            'LSC':px.colors.qualitative.Safe[2],
            'LSD':px.colors.qualitative.Safe[3],
            'Cryo':px.colors.qualitative.Safe[4]
            }

if Restart==0: #if the logging is not being restarted .
    #Get Lakeshore temperature data for the first time
    filename_base=today.strftime("%m_%d_%Y")
    myinstrument.device_serial.open()
    df_read_lakeshore=pd.DataFrame(get_temp_data())
    myinstrument.device_serial.close()    
    #Get cryo temperature data for the first time
    ser0=start_connection(cryo_port)
    df_cryo = cryo_temp_single_read(ser0)
    stop_connection(ser0)

start_time=datetime.datetime.now()
cryo_path=f"Cryocooler_Templog_{filename_base}.csv"
lakeshore_path=f"Lakeshore_Templog_{filename_base}.csv"
webpath =f"Exp_Templog_{filename_base}.html"
# Use this function to search for any files which match your filename
files_present = glob.glob(cryo_path)+glob.glob(lakeshore_path)
# if no matching files, write to csv, if there are matching files, print statement
if files_present:
       df_cryo=pd.read_csv(cryo_path)
       df_read_lakeshore=pd.read_csv(lakeshore_path)

log_duration = 24*60*60 #seconds
log_step=1 #seconds 
time_check_state=start_time

while datetime.datetime.now()-start_time<timedelta(seconds=log_duration):
    # print("its true")
    timer_save_data=datetime.datetime.now()
    timer_plot_data=timer_save_data
    
    while datetime.datetime.now()-timer_save_data<timedelta(seconds=60):
            
        myinstrument=lakeshore_initialize()
        dftemp=pd.DataFrame(get_temp_data())
        myinstrument.device_serial.close() 
        
        
        ser0=start_connection(cryo_port)
        dftemp_cryo=cryo_temp_single_read(ser0)
        stop_connection(ser0)

        df_read_lakeshore=df_read_lakeshore.append(dftemp,ignore_index = True)
        df_cryo=df_cryo.append(dftemp_cryo,ignore_index = True)
        #display(df[-5:])
        df_cryo.to_csv(cryo_path)
        df_read_lakeshore.to_csv(lakeshore_path)
        sleep(60)
        #clear_output(wait=True)    
       
    if datetime.datetime.now()-timer_plot_data>timedelta(seconds=60):
        clear_output(wait=True)
        df_read_cryo=pd.read_csv(cryo_path)
        df_read_lakeshore=pd.read_csv(lakeshore_path)
        # df=df.append(df_read_lakeshore,ignore_index=True)

    fig = plotly.subplots.make_subplots(rows=1, cols=1, vertical_spacing=0.1)
    fig['layout']['margin'] = {'l': 40, 'r': 20, 'b': 40, 't': 100}
    # fig['layout']['legend'] = {'x': 0, 'y': -0.25, 'xanchor': 'left','orientation':"h"}

    fig.append_trace({
        'x': df_read_lakeshore.time,
        'y': df_read_lakeshore.tempA_log,
        'name': f"{Sensor_map['LSA']} @ {np.round(df_read_lakeshore.tempA_log.values[-1],3)} K",
        'type': 'scatter',
        'legendgroup' : 1, 
        'line':dict(color=Color_map['LSA']),
        'hovertemplate':
            f"<b>{Sensor_map['LSA']}</b><br><br>" +
            "Temperature: %{y} K<br>" +
            "Time: %{x}<br>" +
            "<extra></extra>",
    }, 1, 1)

    fig.append_trace({
        'x': df_read_lakeshore.time,
        'y': df_read_lakeshore.tempB_log,
        'name': f"{Sensor_map['LSB']} @ {np.round(df_read_lakeshore.tempB_log.values[-1],3)} K",
        'type': 'scatter',
        'line':dict(color=Color_map['LSB']),  
        'legendgroup' : 1,
        'hovertemplate':
            f"<b>{Sensor_map['LSB']}</b><br><br>" +
            "Temperature: %{y} K<br>" +
            "Time: %{x}<br>" +
            "<extra></extra>",    

    }, 1, 1)

    fig.append_trace({
        'x': df_read_lakeshore.time,
        'y': df_read_lakeshore.tempC_log,
        'name': f"{Sensor_map['LSC']} @ {np.round(df_read_lakeshore.tempC_log.values[-1],3)} K",
        'type': 'scatter',
        'legendgroup' : 2,
        'line':dict(color=Color_map['LSC']),
        'hovertemplate':
            f"<b>{Sensor_map['LSC']}</b><br><br>" +
            "Temperature: %{y} K<br>" +
            "Time: %{x}<br>" +
            "<extra></extra>",  
    }, 1, 1)

    fig.append_trace({
        'x': df_read_lakeshore.time,
        'y': df_read_lakeshore.tempD_log,
        'name': f"{Sensor_map['LSD']} @ {np.round(df_read_lakeshore.tempD_log.values[-1],3)} K",
        'type': 'scatter',
        'legendgroup' : 2,
        'line':dict(color=Color_map['LSD']),
        'hovertemplate':
            f"<b>{Sensor_map['LSD']}</b><br><br>" +
            "Temperature: %{y} K<br>" +
            "Time: %{x}<br>" +
            "<extra></extra>",
    }, 1, 1)

    fig.append_trace({
        'x': df_read_cryo.time,
        'y': df_read_cryo.cryo_temp,
        'name': f"{Sensor_map['Cryo']} @ {np.round(df_read_cryo.cryo_temp.values[-1],3)} K",
        'type': 'scatter',
        'legendgroup' : 3,
        'line':dict(color=Color_map['Cryo']),
        'hovertemplate':
            f"<b>{Sensor_map['Cryo']}</b><br><br>" +
            "Temperature: %{y} K<br>" +
            "Time: %{x}<br>" +
            "<extra></extra>",
    }, 1, 1)
    fig.update_layout(width=1000,height=800)
    xmin=100
    xmax=300
    fig.update_yaxes(range=[xmin,xmax],row=1,col=1)

    t0=datetime.datetime.strptime(df_read_lakeshore.time[0],"%Y-%m-%d %H:%M:%S.%f")
    tstring=t0.strftime("%m/%d/%Y, %H:%M:%S")
    fig.update_layout(
        title={
            'text':f"Detector Cooling Chain Test, Date:{tstring} ",
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        
        yaxis_title = 'Temperature (K)',
        xaxis_title='Time (MST)'

        )

    fig.layout.template='simple_white'
    fig.update_xaxes(showgrid=True,mirror=True)
    fig.update_yaxes(showgrid=True,mirror=True)
    fig.update_layout(legend=dict(  
                                    #title=r"Sensor lables (Click to toggle visiblity)<br>",
                                    groupclick="toggleitem",
                                    orientation="h",
                                    yanchor="top",
                                    y=-0.5,
                                    xanchor="left",
                                    x=0.0
                                )
                    )
    fig.update_xaxes(ticks="outside", tickwidth=1.5, tickcolor='grey', ticklen=10)
    fig.update_yaxes(ticks="outside", tickwidth=1.5, tickcolor='grey', ticklen=10)
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=15,
                        label="15 mins",
                        step="minute",
                        stepmode="backward"),
                    dict(count=1,
                        label="1 hour",
                        step="hour",
                        stepmode="backward"),
                    dict(count=5,
                        label="5 hours",
                        step="hour",
                        stepmode="backward"),
                    dict(count=10,
                        label="10 hours",
                        step="hour",
                        stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=True
            ),
            type="date"
        )
    )
    fig.update_xaxes(rangeslider_bordercolor='grey',rangeslider_borderwidth=1)
    fig.update_xaxes(showspikes=True,spikethickness=1)
    fig.update_yaxes(showspikes=True,spikethickness=1)
    fig.update_layout(
        yaxis = dict(
            tickmode = 'linear',
            dtick = 25
        ))
    fig.show()
    tsave=t0.strftime("%m%d%Y")
    fig.write_html(f"Detector_cooling_test_{tsave}.html")
    fig.write_image(f"Detector_cooling_test_{tsave}.png")

connection successful
Connected
Connection stopped


KeyboardInterrupt: 

In [ ]:
# from IPython.display import clear_output
# # import datetime
# import pandas as pd
# import time 

# today=datetime.now()
# filename_base=today.strftime("%m_%d_%Y")
# cryo_path=f"Exp_Cryocooler_Templog_{filename_base}.csv"
# lakeshore_path=f"Exp_Lakeshore_Templog_{filename_base}.csv"
# html_path=f"Exp_Templog_{filename_base}.html"

# Restart=1 #Raise flag if the logging is being restarted and the data in the exisiting dataframe must not be re written
# Sensor_map={'LSA':'LS Input A: EMCCD Detector',
#             'LSB':'LS Input B: Cold Finger 1',
#             'LSC':'LS Input C: Cold Finger 2',
#             'LSD':'LS Input D: Vapour Shroud',
#             'Cryo':'Cryo Cold Tip'
#             }

# if Restart==0: #if the logging is not being restarted .

#     #Get Lakeshore temperature data for the first time
#     myinstrument.device_serial.open()
#     df=pd.DataFrame(get_temp_data())
#     myinstrument.device_serial.close()    
#     #Get cryo temperature data for the first time
#     ser0=start_connection(cryo_port)
#     df_cryo = cryo_temp_single_read(ser0)
#     stop_connection(ser0)

    
    

# ser0=start_connection(cryo_port)
# current_temp_cryo=get_temperature(ser0)['cryo_temp']
# stop_connection(ser0)
# if current_temp_cryo<modeswitch_temperature:
#     ser0=start_connection(cryo_port)
#     set_target_temperature(coldtip_temperature)
#     set_pid_mode(2)
#     stop_connection(ser0)
#     myinstrument.command(f'SETP 1,{cf1_temperature}')
#     myinstrument.device_serial.close()    

# start_time=datetime.now()

# log_duration = 24*60*60 #seconds
# log_step=1 #seconds 
    
# while datetime.now()-start_time<timedelta(seconds=log_duration):
#     # print("its true")
#     timer_save_data=datetime.now()
#     timer_plot_data=timer_save_data
#     while datetime.now()-timer_save_data<timedelta(seconds=60):
            
#         myinstrument=lakeshore_initialize()
#         dftemp=pd.DataFrame(get_temp_data())
#         myinstrument.device_serial.close() 
        
        
#         ser0=start_connection(cryo_port)
#         dftemp_cryo=cryo_temp_single_read(ser0)
#         stop_connection(ser0)

#         df=df.append(dftemp,ignore_index = True)
#         df_cryo=df_cryo.append(dftemp_cryo,ignore_index = True)
#         #display(df[-5:])
#         df_cryo.to_csv(cryo_path)
#         df.to_csv(lakeshore_path)
#         sleep(20)
#         #clear_output(wait=True)
        
#     if datetime.now()-timer_plot_data>timedelta(seconds=60):
#         clear_output(wait=True)
#         df_read_cryo=pd.read_csv(cryo_path)
#         df_read_lakeshore=pd.read_csv(lakeshore_path)
#         # df=df.append(df_read_lakeshore,ignore_index=True)
        
#         fig = plotly.subplots.make_subplots(rows=3, cols=2, vertical_spacing=0.2)
#         fig['layout']['margin'] = {'l': 30, 'r': 10, 'b': 30, 't': 100}
#         fig['layout']['legend'] = {'x': 0, 'y': -0.25, 'xanchor': 'left','orientation':"h"}

#         fig.append_trace({
#             'x': df_read_lakeshore.time,
#             'y': df_read_lakeshore.tempA_log,
#             'name': Sensor_map['LSA'],
#             'type': 'scatter'
#         }, 1, 1)


#         fig.append_trace({
#             'x': df_read_lakeshore.time,
#             'y': df_read_lakeshore.tempB_log,
#             'name': Sensor_map['LSB'],
#             'type': 'scatter'
#         }, 1, 2)

#         fig.append_trace({
#             'x': df_read_lakeshore.time,
#             'y': df_read_lakeshore.tempC_log,
#             'name': Sensor_map['LSC'],
#             'type': 'scatter'
#         }, 2, 1)

#         fig.append_trace({
#             'x': df_read_lakeshore.time,
#             'y': df_read_lakeshore.tempD_log,
#             'name': Sensor_map['LSD'],
#             'type': 'scatter'
#         }, 2, 2)
        
#         fig.append_trace({
#             'x': df_read_cryo.time,
#             'y': df_read_cryo.cryo_temp,
#             'name': Sensor_map['Cryo'],
#             'type': 'scatter'
#         }, 3, 1)
#         # fig.update_layout(yaxis = dict(range=[77,300]))
#         fig.update_layout(width=900,height=800)

#         fig.update_yaxes(range=[70.0,300.0],row=1,col=1)
#         fig.update_yaxes(range=[70,300],row=1,col=2)
#         fig.update_yaxes(range=[70,300],row=2,col=1)
#         fig.update_yaxes(range=[70,300],row=2,col=2)
#         fig.update_yaxes(range=[70,300],row=3,col=1)
        
#         fig.update_layout(
#             title={
#                 'text':"Detector cooling chain test: Temperature vs time",
#                 'y':0.95,
#                 'x':0.5,
#                 'xanchor': 'center',
#                 'yanchor': 'top'})
#         # fig.update_layout(xaxis_tickformat = '%H:%M')
#         # fig.update_xaxes(ticks= "outside",
#         #                  ticklabelmode= "period", 
#         #                  tickcolor= "black", 
#         #                  ticklen=10, 
#         #                  dtick=10,
#         #                 #  minor=dict(
#         #                 #      ticklen=4,  
#         #                 #      dtick=7*24*60*60*1000,  
#         #                 #      tick0="2016-07-03", 
#         #                 #      griddash='dot', 
#         #                 #      gridcolor='white'),
#         #                 row=1,col=1)
#         fig.layout.template='simple_white'
#         fig.show()
#         fig.write_html(html_path)

In [ ]:
myinstrument.device_serial.close() 

In [ ]:
myinstrument.device_serial.close() 

myinstrument.device_serial.open()
get_temp_data()
myinstrument.device_serial.close() 
myinstrument.device_serial.open()
get_temp_data()

In [ ]:
ser0=start_connection(cryo_port)

In [ ]:
get_temperature(ser0)
stop_connection(ser0)


In [ ]:
ser0.close()

In [ ]:
    # if datetime.now()-time_check_state>timedelta(seconds=60):
    #     time_check_state=datetime.now()
    #     if cooling==True: 
    #         ser0=start_connection(cryo_port)
    #         current_temp_cryo=get_temperature(ser0)['cryo_temp']
    #         stop_connection(ser0)
    #         myinstrument.device_serial.open()
    #         df_lakeshore=pd.DataFrame(get_temp_data())
    #         myinstrument.device_serial.close() 
    #         if current_temp_cryo>cryo_temp_steps[0]:
    #             if  current_temp_cryo<cryo_temp_switchover[0]:
    #                 ser0=start_connection(cryo_port)
    #                 set_target_temperature(cryo_temp_steps[0])
    #                 set_pid_mode(2)
    #                 stop_connection(ser0)
    #             else: 
    #                 ser0=start_connection(cryo_port)
    #                 set_target_power_output(30)
    #                 set_pid_mode(0)
    #                 stop_connection(ser0) 
    #         elif current_temp_cryo>cryo_temp_steps[1]:
    #             if  current_temp_cryo<cryo_temp_switchover[1]:
    #                 ser0=start_connection(cryo_port)
    #                 set_target_temperature(cryo_temp_steps[1])
    #                 set_pid_mode(2)
    #                 stop_connection(ser0)
    #             else: 
    #                 ser0=start_connection(cryo_port)
    #                 set_target_power_output(30)
    #                 set_pid_mode(0)
    #                 stop_connection(ser0)
    #                 myinstrument.device_serial.open()
    #                 myinstrument.command(f'SETP 1,{cf1_temperature}')
    #                 myinstrument.query(f'RANGE 1,5')
    #                 myinstrument.device_serial.close()           
    #         elif current_temp_cryo>cryo_temp_steps[2]:
    #             if  current_temp_cryo<cryo_temp_switchover[2]:
    #                 ser0=start_connection(cryo_port)
    #                 set_target_temperature(cryo_temp_steps[2])
    #                 set_pid_mode(2)
    #                 stop_connection(ser0)
    #     else: 
    #         ser0=start_connection(cryo_port)
    #         set_soft_stop_state(1)
    #         stop_connection(ser0)
    #         myinstrument.device_serial.open()
    #         myinstrument.command(f'SETP 1,285')
    #         myinstrument.device_serial.close()
    #         myinstrument.device_serial.open()
    #         myinstrument.command(f'RANGE 1,5')
    #         myinstrument.device_serial.close() 